In [1]:
from torchtext import datasets, data
import numpy as np
import os, sys
from time import time

from neural_interaction_detection import *
from sampling_and_inference import *
from utils.general_utils import *
from utils.text_utils import *

import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2

device = torch.device("cpu")

/home/aidan/.pyenv/versions/3.6.8/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/aidan/.pyenv/versions/3.6.8/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/aidan/.pyenv/versions/3.6.8/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/aidan/.pyenv/versions/3.6.8/lib/python3.6/s

## Load Model

In [2]:
model = get_bert_model(device)

Widget Javascript not detected.  It may not be installed or enabled properly.


## Classify Sentence

In [4]:
sentence = "this was not a great movie, but a good movie nevertheless"

out = model(sentence)
pred = np.argmax(out[0])
print(("positive" if pred== 1 else "negative") + " sentiment")

positive sentiment


## Run MADEX

In [5]:
data_inst = {"orig": sentence}
Xs, Ys = generate_perturbation_dataset_text(data_inst, model, 1, device, model_id="bert", batch_size=500, seed=42, std_scale=True)

100%|██████████| 12/12 [02:27<00:00, 12.08s/it]


In [5]:
t0 = time()
interactions, mlp_loss = detect_interactions(Xs, Ys, detector="GradientNID", add_linear=True, device=device, weight_samples=True, seed=42, verbose=False)
print("{} test loss, {} seconds elapsed".format(round(mlp_loss, 4), round(time() - t0, 1)))

0.0142 test loss, 29.9 seconds elapsed


## Show Main Effects and Interaction Interpretations

In [6]:
print(sentence + "\n")

dom_map = data_inst["domain_mapper"]

lime_atts = get_lime_attributions(Xs, Ys)
print("main effects:", map_words([i for i, a in lime_atts if a*(pred*2-1) > 0], dom_map))

print("\ntop-5 interactions")
for i, inter_tuple in enumerate(interactions[:5]):
    inter, strength = inter_tuple
    word_inter = map_words(inter, dom_map)
    print("inter {}:".format(i+1), word_inter, strength)

this was not a great movie, but a good movie nevertheless

main effects: ('but', 'a', 'good', 'movie', 'nevertheless')

top-5 interactions
inter 1: ('not', 'but') 2.7557428
inter 2: ('but', 'good') 1.9747727
inter 3: ('not', 'good') 1.8207084
inter 4: ('great', 'good') 1.3452238
inter 5: ('not', 'great') 1.2503706
